In [1]:
import pandas as pd

In [147]:
import json
data = {}
with open('homework.json') as f:
    i = 0
    for line in f:
        d = json.loads(line)
        data[i] = d
        i+=1


In [148]:
df = pd.DataFrame.from_dict(data, orient = 'index')
df = df.drop(columns = ['emotion_9']) #always encoded as 1. drop

In [150]:
df.head()

,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,emotion_5,emotion_6,emotion_7,emotion_8,headline,summary,worker_id
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Parenting Lessons From a Partial Eclipse,I don’t do partiality. Maybe it was finally ti...,95524929
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,On Catalonia,Catalan lawmakers declared independence from S...,30266977
2,0.0,0.0,-2,0.0,0.0,0.0,0.0,0.0,0.0,A Spark of Romance Under Fluorescent Lighting,"Rose Burlingham, an art dealer and writer, mar...",54197764
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,Apple Removes Apps From China Store That Help ...,The world’s most valuable company appears to h...,41177265
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Commerce Secretary’s Offshore Ties to Putin ‘C...,"Wilbur Ross, the commerce secretary, retained ...",56518865


In [157]:
# remove weird boolean rows
df['drop'] = False
for i in range(9):
    df[f'emotion_{i}'] = pd.to_numeric(df[f'emotion_{i}'],errors = 'coerce')
    df.loc[df[f'emotion_{i}'].isin([0.,1.])==False,"drop"] = True

# get rid of dumb text
drops = ["fnord","-1","cat","-2",""]
df.loc[df[f'headline'].isin(drops),"drop"] = True
df.loc[df[f'summary'].isin(drops),"drop"] = True
df = df.loc[df['drop']==False]

#sum ratings on story level
df2 = df.drop(columns = ["drop","worker_id"]).groupby(["headline","summary"]).sum().reset_index()
sc = df2['summary'].value_counts().reset_index()
summary_drops = list(sc.loc[sc["summary"]>1,"index"]) #only keep single summary-headline
df3 = df2.loc[~df2['summary'].isin(summary_drops)] # remove duplicates

#get modal rating for story
rename_dic = dict(zip([f"emotion_{x}" for x in range(9)],[x for x in range(9)]))
df3 = df3.rename(columns = rename_dic)
df3['emotion']=df3[[x for x in range(9)]].idxmax(axis=1)

In [159]:

df3.head()

,headline,summary,0,1,2,3,4,5,6,7,8,emotion
0,#MeToo Floods Social Media With Stories of Har...,"Many women, as well as men, are using the hash...",0.0,0.0,0.0,1.0,4.0,2.0,0.0,7.0,2.0,7
1,"$1.3 Million Homes in Oregon, Michigan and Cal...",A Dwell-style home in Portland; a 1929 Tudor h...,1.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,5.0,8
2,$1.3 Million for That?,Whatever Paul Manafort may or may not have don...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,8
3,"$1.5 Million Homes in California, Montana and ...","A floor-through condominium in San Francisco, ...",2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,5.0,8
4,"$1.5 Million Homes in Rhode Island, California...","An 18th-century farmhouse outside Newport, a h...",0.0,0.0,2.0,0.0,1.0,8.0,2.0,0.0,1.0,5


In [160]:
df3.to_csv("clean.csv",index = False)
